## Chapter 2 - Causal Diagram and Conditional Independence

<a target="_blank" href="https://colab.research.google.com/github/CausalAILab/causalai-book/blob/main/ch2_causal-diagram.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Recall the definition of Causal Diagram from Section 2.4.1.

### Definition 2.4.1 — Causal Diagram
Consider an SCM $\mathbf{M} = \langle \mathbf{U}, \mathbf{V}, \mathcal{F}, P(\mathbf{U}) \rangle$. Then, $\mathbf{G}$ is said to be a **causal diagram** (of $\mathbf{M}$) if constructed as follows:

1. Add a vertex for every endogenous variable in the set $\mathbf{V}$.
2. Add a directed edge $(V_j \to V_i)$ for every $V_i, V_j \in \mathbf{V}$ if $V_j$ appears as an argument of $f_i \in \mathcal{F}$.
3. Add a bidirected edge $(V_j \leftrightarrow V_i)$ for every $V_i, V_j \in \mathbf{V}$ if the corresponding $U_i, U_j \subseteq \mathbf{U}$ are correlated or the corresponding functions $f_i, f_j$ share some $U \in \mathbf{U}$ as an argument.

In [1]:
from sympy import symbols

from scm import SymbolicSCM
from causal_diagram import CausalDiagram

#### Theorem 2.4.1 — Bayesian Factorization (Markov-Observational case)

Every Markovian structural causal model $\mathbf{M}$ (recursive, with independent disturbances) induces an observational distribution over the endogenous variables $\mathbf{V}$ that can be factorized as:

$$
P(\mathbf{v}) = \prod_i P(v_i \mid \text{pa}_i),
\tag{2.45}
$$

where $\text{pa}_i$ are the values of the endogenous parents of $v_i$ in the causal diagram \( \mathcal{G} \).

In [2]:
m1 = CausalDiagram(V = ['X', 'Y', 'Z'], directed_edges = [('X', 'Y'), ('X', 'Z')])
m1.bayesian_factorization()

'P(V) = P(X) P(Y | X) P(Z | X)'

#### Definition 2.4.2 — Conditional independence relation

A set of variables $\mathbf{Y}$ is said to be *conditionally independent* of another set of variables $\mathbf{X}$, given a third set $\mathbf{Z}$, if knowing the values of $\mathbf{X}$ does not change the likelihood of $\mathbf{Y}$ given $\mathbf{Z}$. Formally, this statement is written as:

$$
P(\mathbf{Y} = \mathbf{y} \mid \mathbf{X} = \mathbf{x}, \mathbf{Z} = \mathbf{z}) = P(\mathbf{Y} = \mathbf{y} \mid \mathbf{Z} = \mathbf{z}),
\tag{2.53}
$$

for all $\mathbf{x}, \mathbf{y}, \mathbf{z}$. For simplicity, the equation above is also written as $\mathbf{Y} \perp\!\!\!\perp \mathbf{X} \mid \mathbf{Z}$.

#### Example 2.10 - Causal Chain

Consider the following SCM from Example 2.10

$$
M^1 = \begin{cases}
    \mathbf V &= \{X, Y, Z\}\\
    \mathbf U &= \{U_x, U_y, U_z\}\\
    F &= \begin{cases}
        X := U_x \\
        Y := X \lor \lnot U_z \\
        Z := Z \land U_y \\
    \end{cases}\\
    P(\mathbf U) &= \begin{cases}
       U_z &\sim \mathrm{Bern}(0.5) \\
       U_x &\sim \mathrm{Bern}(0.5) \\
       U_y &\sim \mathrm{Bern}(0.5) \\
    \end{cases}
\end{cases}
$$

We may implement this SCM as follows:

In [3]:
z, x, y, uz, ux, uy = symbols('z x y uz ux uy')

m1 = SymbolicSCM(
    f={
        x : ux,
        z : x & ~uz,
        y : z & uy
    },
    pu={
        uz: 0.5,
        ux: 0.5,
        uy: 0.5,
    }
)

m1

SCM([x, z, y], [uz, ux, uy])

In [4]:
m1.query({x:1, y:1}, latex=True)

<IPython.core.display.Latex object>

In [5]:
m1.query({x:1}, latex=True)

<IPython.core.display.Latex object>

In [6]:
m1.query({y:1}, latex=True)

<IPython.core.display.Latex object>

Hence, we see that $P(X = 1, Y = 1) \neq P(X = 1) P(Y = 1)$. Feel free to play around with the values of $P(U_x), P(U_y), P(U_z)$ and the variables to see what happens.

#### Example 2.11 - Common Cause

Consider the following SCM from Example 2.10

$$
M^2 = \begin{cases}
    \mathbf V &= \{X, Y, Z\}\\
    \mathbf U &= \{U_x, U_y, U_z\}\\
    F &= \begin{cases}
        Z := U_z \\
        X := Z \lor \lnot U_x \\
        Y := Z \lor U_y \\
    \end{cases}\\
    P(\mathbf U) &= \begin{cases}
       U_z &\sim \mathrm{Bern}(0.5) \\
       U_x &\sim \mathrm{Bern}(0.5) \\
       U_y &\sim \mathrm{Bern}(0.5) \\
    \end{cases}
\end{cases}
$$

We may implement this SCM as follows:

In [7]:
z, x, y, uz, ux, uy = symbols('z x y uz ux uy')

m2 = SymbolicSCM(
    f={
        z : uz,
        x : z & ~ux,
        y : z & uy
    },
    pu={
        uz: 0.5,
        ux: 0.5,
        uy: 0.5,
    }
)

m2

SCM([z, x, y], [uz, ux, uy])

In [8]:
m2.query({x:1, y:1}, given={z:1}, latex=True)

<IPython.core.display.Latex object>

In [9]:
m2.query({x:1}, given={z:1}, latex=True)

<IPython.core.display.Latex object>

In [10]:
m2.query({y:1}, given={z:1}, latex=True)

<IPython.core.display.Latex object>

Note that $P(X = 1, Y = 1 \mid Z = 1) = P(X = 1 \mid Z = 1) P(Y = 1 \mid Z = 1)$. Feel free to play around with the values of $P(U_x), P(U_y), P(U_z)$ and the the variables to see what happens.

#### Example 2.12 - Common Effect

Consider the following SCM from Example 2.10

$$
M^3 = \begin{cases}
    \mathbf V &= \{X, Y, Z\}\\
    \mathbf U &= \{U_x, U_y, U_z\}\\
    F &= \begin{cases}
        X := U_x, \\
        Y := U_y, \\
        Z := \lnot Y \land (\lnot X \lor U_z) \\
    \end{cases}\\
    P(\mathbf U) &= \begin{cases}
       U_z &\sim \mathrm{Bern}(0.5) \\
       U_x &\sim \mathrm{Bern}(0.5) \\
       U_y &\sim \mathrm{Bern}(0.5) \\
    \end{cases}
\end{cases}
$$

We may implement this SCM as follows:

In [11]:
z, x, y, uz, ux, uy = symbols('z x y uz ux uy')

m3 = SymbolicSCM(
    f={
        x : ux,
        y : uy,
        z : ~y & (~x | uz)
    },
    pu={
        uz: 0.5,
        ux: 0.5,
        uy: 0.5,
    }
)

m3

SCM([x, y, z], [uz, ux, uy])

In [12]:
m3.query({x:1, y:1}, latex=True)

<IPython.core.display.Latex object>

In [13]:
m3.query({x:1}, latex=True)

<IPython.core.display.Latex object>

In [14]:
m3.query({y:1}, latex=True)

<IPython.core.display.Latex object>

We see that $P(X = 1, Y = 1) = P(X = 1) P(Y = 1)$. Let's condition on $Z$ and see what happens. 

In [15]:
m3.query({x:1, y:1}, given={z:0}, latex=True)

<IPython.core.display.Latex object>

In [16]:
m3.query({x:1}, given={z:0}, latex=True)

<IPython.core.display.Latex object>

In [17]:
m3.query({y:1}, given={z:0}, latex=True)

<IPython.core.display.Latex object>

Note that, $P(X = 1, Y = 1 \mid Z = 0) \neq P(X = 1 \mid Z = 1) P(Y = 1 \mid Z = 0)$. Feel free to play around with the values of $P(U_x), P(U_y), P(U_z)$ and the the variables to see what happens.

#### Example 2.13 - Common Effect (Extended)

Consider the following SCM from Example 2.10

$$
M^4 = \begin{cases}
    \mathbf V &= \{X, Y, Z, W\}\\
    \mathbf U &= \{U_x, U_y, U_z, U_w\}\\
    F &= \begin{cases}
        X := U_x, \\
        Y := U_y, \\
        Z := \lnot Y \land (\lnot X \lor U_z) \\
        W := Z \oplus U_w
    \end{cases}\\
    P(\mathbf U) &= \begin{cases}
       U_z &\sim \mathrm{Bern}(0.5) \\
       U_x &\sim \mathrm{Bern}(0.5) \\
       U_y &\sim \mathrm{Bern}(0.5) \\
       U_w &\sim \mathrm{Bern}(0.1) 
    \end{cases}
\end{cases}
$$

We may implement this SCM as follows:

In [18]:
z, x, y, w, uz, ux, uy, uw = symbols('z x y w uz ux uy uw')

m4 = SymbolicSCM(
    f={
        x : ux,
        y : uy,
        z : ~y & (~x | uz),
        w : (uw & ~z) | (~uw & z)
    },
    pu={
        uz: 0.5,
        ux: 0.5,
        uy: 0.5,
        uw: 0.1
    }
)

m4

SCM([x, y, z, w], [uz, ux, uy, uw])

In [19]:
m4.query({x:1, y:1}, given={w:1}, latex=True)

<IPython.core.display.Latex object>

In [20]:
m4.query({x:1}, given={w:1}, latex=True)

<IPython.core.display.Latex object>

In [21]:
m4.query({y:1}, given={w:1}, latex=True)

<IPython.core.display.Latex object>

Note that, $P(X = 1, Y = 1 \mid Z = 0) \neq P(X = 1 \mid Z = 1) P(Y = 1 \mid Z = 0)$. Feel free to play around with the values of $P(U_x), P(U_y), P(U_z)$ and the the variables to see what happens.

#### Example 2.13 - Common Effect (Extended)

Consider the following SCM from Example 2.10

$$
M^5 = \begin{cases}
    \mathbf V &= \{X, Y, Z\}\\
    \mathbf U &= \{U_x, U_y, U_z, U_w\}\\
    F &= \begin{cases}
        X := U_x, \\
        Y := U_y, \\
        Z := \lnot (X \oplus Y)
    \end{cases}\\
    P(\mathbf U) &= \begin{cases}
       U_x &\sim \mathrm{Bern}(0.5) \\
       U_y &\sim \mathrm{Bern}(0.5) \\
    \end{cases}
\end{cases}
$$

We may implement this SCM as follows:

In [22]:
z, x, y, ux, uy = symbols('z x y ux uy')

m5 = SymbolicSCM(
    f={
        x : ux,
        y : uy,
        z : ~((x & ~y) | (~x & y))
    },
    pu={
        ux: 0.5,
        uy: 0.5
    }
)

m5

SCM([x, y, z], [ux, uy])

In [23]:
m5.query({y:1}, given={x:1, z:1}, latex=True)

<IPython.core.display.Latex object>

In [24]:
m5.query({y:1}, given={z:1}, latex=True)

<IPython.core.display.Latex object>

We see that $P(Y = 1 \mid X = 1, Z = 1) \neq P(Y = 1 \mid Z = 1)$. 

#### Definition 2.4.3 — d-separation

Consider a causal diagram $ \mathcal{G} $ and disjoint sets of endogenous variables $ \mathbf{X}, \mathbf{Y}, \mathbf{Z} $. A path $ p $ between $ X \in \mathbf{X} $ and $ Y \in \mathbf{Y} $ in $ \mathcal{G} $ is **blocked** by $ \mathbf{Z} $ if and only if:

1. $ p $ contains a chain $ A \rightarrow B \rightarrow C $ or a fork $ A \leftarrow B \rightarrow C $ such that the middle node $ B $ is in $ \mathbf{Z} $, or

2. $ p $ contains a collider $ A \rightarrow B \leftarrow C $ such that the collision node $ B $ or any of its descendants is not in $ \mathbf{Z} $.

If $ \mathbf{Z} $ blocks every path between every $ X \in \mathbf{X} $ and $ Y \in \mathbf{Y} $ in $ \mathcal{G} $, then the pair $ \mathbf{X}, \mathbf{Y} $ is said to be **d-separated** (or "closed") given $ \mathbf{Z} $. This is written as $ (\mathbf{X} \perp\!\!\!\perp \mathbf{Y} \mid \mathbf{Z})_{\mathcal{G}} $. Otherwise, $ X \in \mathbf{X} $ and $ Y \in \mathbf{Y} $ are said to be **d-connected** ("opened").


![Example Causal Diagrams](Fig-2-16.png)

### Example 2.15 - d-separation

In [25]:
V1 = ['C', 'S', 'R', 'W', 'L']

directed_edges1 = [
    ['C', 'S'],
    ['C', 'R'],
    ['S', 'W'],
    ['R', 'W'],
    ['W', 'L']
]

G1 = CausalDiagram(V1, directed_edges1)

We can get all the paths from one variable to another using the `path` function.

In [26]:
G1.path('S', 'W')

S -> W
S <- C -> R -> W


### Example 2.16 - d-separation (2)

For this example, we will examine the paths between $R$ and $L$ and check whether they are blocked by $W$.

In [27]:
G1.path('R', 'L')

R -> W -> L
R <- C -> S -> W -> L


### Example 2.17 - d-separation in non-Markovian models

Here we will check the paths between $S$ and $L$ for the non-Markovian graph in Fig. 2.16.

In [28]:
V2 = ['S', 'R', 'W', 'L']

directed_edges2 = [
    ['S', 'W'],
    ['R', 'W'],
    ['W', 'L']
]

bidirected_edges2 = [ ['S', 'R'] ]

G2 = CausalDiagram(V2, directed_edges2, bidirected_edges2)

In [29]:
G2.path('S', 'L')

S -> W -> L
S <-> R -> W -> L


### Example 2.18 - d-separation in non-Markovian Models (2)

Here we will look at the graph in Fig. 2.17 and try to find a set that blocks all paths between $X$ and $Y$.

In [30]:
V3 = ['X', 'T', 'W', 'Z', 'R', 'Y', 'S']

directed_edges3 = [
    ['X', 'W'],
    ['T', 'X'],
    ['T', 'Z'],
    ['W', 'Z'],
    ['Z', 'Y'],
    ['R', 'Y'],
    ['Y', 'S']
]

bidirected_edges3 = [['W', 'Y']]

G3 = CausalDiagram(V3, directed_edges3, bidirected_edges3)

In [31]:
G3.path('X', 'Y')

X -> W -> Z -> Y
X -> W <-> Y
X <- T -> Z -> Y
X <- T -> Z <- W <-> Y


Note, $Z = {T}$ will block paths except $X \to W \to Z \to Y$. Blocking it requires $W$ or $Z$ to be in $\mathbf{Z}$, but $X \leftarrow T \to Z \leftarrow W <-> Y$ will be unblocked in both cases. After all, there is no set $\mathbf{Z}$ that d-separates X and Y in this diagram.

### Example 2.20 - Check D-separation algorithm

Algorithm 1 is implemented in `d_separation_check` of `CausalDiagram`. 

In [32]:
G3.d_separation_check(set(['X']), set(['Y']), set(['Z']), verbose=True)

Ancestors of X, Y, Z: {'X', 'W', 'Y', 'R', 'T', 'Z'}
Subgraph: {'X': {'W', 'Y', 'T'}, 'W': {'Y', 'X', 'T'}, 'Y': {'W', 'R', 'X'}, 'R': {'Y'}, 'T': {'W', 'X'}, 'Z': set()}


False